In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.experimental import enable_halving_search_cv
#from sklearn.model_selection import HalvingRandomSearchCV


In [2]:
test_run = True
#test_run = False

In [3]:
df = pd.read_csv("Temp/df_proc_train_set.csv")

In [4]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
if test_run:
    df0 = pd.DataFrame()
    #index = 0
    for index in tqdm(df.index):
        if index % 100 == 0:
            #row_i = df.loc[index, :]
            df0 = pd.concat([df0, df.loc[index, :]], axis=1) #, row_i])
    df0 = df0.T
    df0['sol_category'].value_counts()
    df = df0

100%|██████████| 70711/70711 [00:11<00:00, 5949.63it/s] 


In [6]:
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100,EOS28121,O=C(Nc1ccccc1F)N1CCC(n2cnc3cc(F)ccc3c2=O)CC1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,EOS2381,Cc1c2c(=O)n(-c3nc4ccccc4s3)[nH]c2cc(=O)n1Cc1cccs1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
300,EOS100352,N[C@@H](COP(=O)(O)O)C(=O)O,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,EOS75800,CC1(C)C[C@H](C(=O)NC2CCN(Cc3cccnc3)CC2)C(C)(C)O1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,EOS93381,Cc1ncc(CNC(=O)c2ccc(NC(C)C)nc2)c(N)n1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70400,EOS71926,Cc1ccnc(NC(=O)CN(C)C(=O)[C@H](C)c2c(F)cccc2F)c1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70500,EOS13092,Cc1cc(C(=O)Nc2cccc(C(=O)NCc3ccccc3)c2)no1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70600,EOS94840,CCOc1ccccc1N1CCN(C(=O)[C@@H]2COCCN2)CC1.Cl.Cl,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#y = pd.read_excel(xl, 'y', header = None)

In [8]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [9]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

708


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
300,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70400,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70500,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70600,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Only exfp

In [10]:
X_ecfp = df.copy(deep=True)
drop_list_desc = [column for column in X_ecfp.columns if (column.startswith('example_') or column.startswith('rdkit_desc_'))]

# Drop features 
X_ecfp.drop(drop_list_desc, axis=1, inplace=True)
X_ecfp.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_ecfp

,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,ecfp_4086,ecfp_4087,ecfp_4088,ecfp_4089,ecfp_4090,ecfp_4091,ecfp_4092,ecfp_4093,ecfp_4094,ecfp_4095
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Only descriptors

In [11]:
X_desc = df.copy(deep=True)
drop_list_MFP = [column for column in X_desc.columns if column.startswith('ecfp_')]

# Drop features 
X_desc.drop(drop_list_MFP, axis=1, inplace=True)
X_desc.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_desc

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100,3.5437,101.2237,67.23,6,1,2,4,2,8,46,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,3.50842,109.4917,72.68,6,1,3,7,0,8,41,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
300,-1.4924,33.6073,130.08,7,5,4,4,0,8,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,2.7559,98.0287,54.46,5,1,4,4,1,5,56,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,1.51252,85.3473,105.82,7,4,5,6,1,7,42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70400,2.86882,90.0137,62.3,5,1,5,3,2,7,44,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70500,3.16532,93.2474,84.23,6,2,5,4,2,6,42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70600,0.7223,89.0137,54.04,6,1,4,5,1,6,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### only example descriptors

In [12]:
X_example = X_desc.copy(deep=True)
drop_list_ex = [column for column in X_example.columns if column.startswith('rdkit_desc_')]

# Drop features 
X_example.drop(drop_list_ex, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_example

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,example_NumSaturatedRings,example_NumAliphaticRings,example_NumAromaticHeterocycles,example_NumSaturatedHeterocycles,example_NumAliphaticHeterocycles,example_NumAromaticCarbocycles,example_NumSaturatedCarbocycles,example_NumAliphaticCarbocycles,example_FractionCSP3,example_HallKierAlpha
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,1,1,2,1,1,1,0,0,0.263158,-3.56
100,3.5437,101.2237,67.23,6,1,2,4,2,8,46,...,1,1,1,1,1,2,0,0,0.25,-3.29
200,3.50842,109.4917,72.68,6,1,3,7,0,8,41,...,0,0,4,0,0,1,0,0,0.105263,-2.97
300,-1.4924,33.6073,130.08,7,5,4,4,0,8,19,...,0,0,0,0,0,0,0,0,0.666667,-0.46
400,2.7559,98.0287,54.46,5,1,4,4,1,5,56,...,2,2,1,2,2,0,0,0,0.7,-1.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,1.51252,85.3473,105.82,7,4,5,6,1,7,42,...,0,0,2,0,0,0,0,0,0.333333,-2.7
70400,2.86882,90.0137,62.3,5,1,5,3,2,7,44,...,0,0,1,0,0,1,0,0,0.277778,-2.83
70500,3.16532,93.2474,84.23,6,2,5,4,2,6,42,...,0,0,1,0,0,2,0,0,0.105263,-3.41
70600,0.7223,89.0137,54.04,6,1,4,5,1,6,48,...,2,2,0,2,2,1,0,0,0.588235,-1.79


#### only rdkit descriptors

In [13]:
X_rdd = X_desc.copy(deep=True)
drop_list_rd = [column for column in X_rdd.columns if column.startswith('example_')]

# Drop features 
X_rdd.drop(drop_list_rd, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_rdd

,rdkit_desc_MaxEStateIndex,rdkit_desc_MinEStateIndex,rdkit_desc_MinAbsEStateIndex,rdkit_desc_qed,rdkit_desc_MaxPartialCharge,rdkit_desc_MinPartialCharge,rdkit_desc_MaxAbsPartialCharge,rdkit_desc_FpDensityMorgan1,rdkit_desc_FpDensityMorgan2,rdkit_desc_FpDensityMorgan3,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,13.75148,-0.805113,0.046085,0.732517,0.321549,-0.352889,0.352889,1.068966,1.827586,2.517241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100,13.695895,-0.489849,0.104663,0.734897,0.321426,-0.324377,0.324377,1.107143,1.857143,2.607143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
200,13.078106,-0.193069,0.133241,0.508949,0.282478,-0.306754,0.306754,1.148148,2.0,2.777778,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
300,9.982029,-4.602616,0.708719,0.392075,0.469148,-0.480021,0.480021,1.636364,2.090909,2.363636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,12.776512,-0.394251,0.07322,0.911603,0.225833,-0.368848,0.368848,1.32,2.04,2.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,12.076581,-0.218915,0.218915,0.772645,0.252617,-0.383303,0.383303,1.272727,2.045455,2.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70400,13.814185,-1.045283,0.255092,0.904614,0.244635,-0.335914,0.335914,1.24,1.88,2.48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70500,12.27816,-0.390098,0.191141,0.749936,0.277351,-0.360912,0.360912,1.0,1.8,2.52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70600,12.505976,-0.188879,0.15504,0.886307,0.241722,-0.491708,0.491708,1.304348,2.086957,2.782609,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y = df.iloc[:, 2]

In [15]:
#y

In [16]:
y.unique()

array([0, 1, 2], dtype=object)

## StandardScaler

In [17]:
#scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
#X = scaler.fit_transform(X)

## split

In [18]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1,  shuffle=True, stratify=y)

In [19]:
# find out where NaN is, remove in processing

## multi-class logistic regression

In [20]:
#clf = LogisticRegression(random_state=0, max_iter=1000, verbose=1).fit(X_train, y_train)

In [21]:
#clf.predict(X[:2, :])

In [22]:
#clf.predict_proba(X[:2, :])

In [23]:
#y_prediction = clf.predict(X_test)

In [24]:
#df_y = pd.DataFrame(y_prediction)

In [25]:
# Stricter feature selection
### 1. just use ecfp/rdkit+example
### 2. Hyperparatuning (StratifiedKfold+RandomizedSearchCV on X_train and y_train) (max_iter, solver, penalty,C)

# for dataset over all_dataset
#   hyperparameter tuning()

In [26]:
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from numpy import linspace

In [27]:
model = LogisticRegression(random_state=0)

parameters = {
    'max_iter' : [2000], #range(1000, 2000),
    #'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    #'penalty' : ['l2', 'none'],
    'C' : linspace(0.1, 1.0, num=50)
    }

In [28]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [29]:
all_dataset = list()
all_dataset.append(X)
all_dataset.append(X_ecfp)
all_dataset.append(X_desc)
#all_dataset.append(X_example)
#all_dataset.append(X_rdd)

In [30]:
# setup parameters for xgboost
param = {}
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param["eval_metric"] = "error"
param['eta'] = 0.3
param['gamma'] = 0
param['max_depth'] = 6
param['min_child_weight']=1
param['max_delta_step'] = 0
param['subsample']= 1
param['colsample_bytree']=1
param['silent'] = 1
param['seed'] = 0
param['base_score'] = 0.5



In [32]:
#qwck_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

print('------------------------------------------------------')

for dataset in all_dataset:
    # apply StandardScaler
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)
    dataset = scaler.fit_transform(dataset)

    # split dataset
    StratifiedKFold(n_splits=5)
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, random_state=1,  shuffle=True, stratify=y)#, test_size=0.2, train_size=0.8)

    # method
    clf = XGBClassifier(**param)
    metLearn=CalibratedClassifierCV(clf, method='sigmoid', cv=5, n_jobs=5)
    metLearn.fit(X_train, y_train)
    testPredictions = metLearn.predict(X_test)

    # output
    #print(f'results of data set {dataset}')
    #print('------------------------------------------------------')
    #print(" Results from Random Search: " )
    #print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
    #print("\n The best score across ALL searched params:\n", randm_src.best_score_)
    #print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)
    #print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')



------------------------------------------------------


ValueError: Unknown label type: 'unknown'

## Score

In [ ]:
#sk_cohen_kappa = cohen_kappa_score(y_prediction, y_test, weights='quadratic')
#print(f'quadratic cohen kappa: {sk_cohen_kappa}')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#confusion_matrix(y_test, y_prediction)

In [ ]:
#df_y[0].value_counts()

In [ ]:
#len(y_test)